In [1]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import caffe
import itertools as it
import pandas as pd
import cv2
import scipy
import seaborn
import lmdb
import shutil

from caffe.proto import caffe_pb2
from collections import defaultdict
from math import ceil
from copy import copy
from collections import OrderedDict
from operator import itemgetter, attrgetter
from matplotlib import pyplot
from scipy import ndimage as ndimage
from os.path import join as pjoin

from caffe import layers as L
from caffe import params as P
import tools

from ntb.layer.data import Transformer
from ntb.db import *

from training.util import *

%matplotlib inline
pyplot.rcParams['figure.figsize'] = (10, 6)

In [2]:
NET_NAME = "raw_50_ft_googlenet_adam"
NET_DIR = pjoin(NETS_DIR, NET_NAME)

DATA_DIR = pjoin(NET_DIR, 'data')
SNAPSHOTS_DIR = pjoin(NET_DIR, 'snapshots')
LOGS_DIR = pjoin(NET_DIR, 'logs')
DATASET_DIR = pjoin(DATASETS_DIR, 'raw_50')
LABELS = np.load(pjoin(DATASET_DIR, 'labels.np'))

### Create all folders

In [3]:
for net_dir in (DATA_DIR, SNAPSHOTS_DIR, LOGS_DIR):
    if not os.path.isdir(net_dir):
        os.makedirs(net_dir)

# Training

In [3]:
caffe.set_mode_gpu()

### Setup logging

In [4]:
logs_file_path = pjoin(LOGS_DIR, 'train.log')
os.environ['GLOG_log_dir'] = LOGS_DIR
os.environ['GLOG_logbufsecs'] = "1"
caffe.set_log_info(logs_file_path)

### Write nets and solver files

In [5]:
solverprototxt = tools.CaffeSolver(
    trainnet_prototxt_path=os.path.join(NET_DIR, "trainnet.prototxt"),
    testnet_prototxt_path=os.path.join(NET_DIR, "valnet.prototxt"),
)
solverprototxt.sp['solver_mode'] = 'GPU'
solverprototxt.sp['base_lr'] = "0.0001"
solverprototxt.sp['display'] = "1"
solverprototxt.sp['snapshot'] = "500"
solverprototxt.sp['lr_policy'] = '"fixed"'
solverprototxt.sp['test_interval'] = "200"
solverprototxt.sp['test_iter'] = "8600"
solverprototxt.sp['max_iter'] = "40000"
solverprototxt.sp['momentum'] = "0.9"
solverprototxt.sp['snapshot_prefix'] = '"{}"'.format(os.path.join(SNAPSHOTS_DIR, 'snapshot'))
solverprototxt.write(os.path.join(NET_DIR, 'solver.prototxt'))

### Load solver

In [6]:
solver = caffe.AdamSolver(os.path.join(NET_DIR, 'solver.prototxt'))

### Start new training

In [8]:
solver.net.copy_from('/storage/models/googlenet/bvlc_googlenet.caffemodel')
# solver.test_nets[0].share_with(solver.net)

### Restore from snapshot

In [8]:
for snapshot in sorted(os.listdir(SNAPSHOTS_DIR)):
    if snapshot.endswith('solverstate'):
        print snapshot

snapshot_iter_100.solverstate
snapshot_iter_1000.solverstate
snapshot_iter_10000.solverstate
snapshot_iter_1500.solverstate
snapshot_iter_2000.solverstate
snapshot_iter_2500.solverstate
snapshot_iter_3000.solverstate
snapshot_iter_3500.solverstate
snapshot_iter_4000.solverstate
snapshot_iter_4500.solverstate
snapshot_iter_500.solverstate
snapshot_iter_5000.solverstate
snapshot_iter_5500.solverstate
snapshot_iter_6000.solverstate
snapshot_iter_6500.solverstate
snapshot_iter_7000.solverstate
snapshot_iter_7500.solverstate
snapshot_iter_8000.solverstate
snapshot_iter_8500.solverstate
snapshot_iter_9000.solverstate
snapshot_iter_9500.solverstate


In [7]:
solver.restore(os.path.join(SNAPSHOTS_DIR, 'snapshot_iter_30000.solverstate'))
solver.test_nets[0].share_with(solver.net)

### Check images and labels

In [37]:
# transformer = Transformer(shape=[227, 227])
# image_index = 3
# pyplot.figure()
# pyplot.imshow(transformer.deprocess(copy(solver.net.blobs['data'].data[image_index, ...])))
# gt = solver.net.blobs['label'].data[image_index, ...].astype(np.int)
# pyplot.title('GT: {}'.format(labels[gt]))
# pyplot.axis('off');

### Train network

In [8]:
%%time
solver.solve()

CPU times: user 1h 37min 48s, sys: 2h 18min 5s, total: 3h 55min 53s
Wall time: 2h 52min 26s


### Rename log file

In [13]:
newest_log = pjoin(LOGS_DIR, max(os.listdir(LOGS_DIR), key=lambda log:  os.path.getctime(pjoin(LOGS_DIR, log))))
print newest_log, '->', logs_file_path
shutil.copyfile(newest_log, logs_file_path)
!sed -i "s/ (.*),/,/" $logs_file_path

/storage/nets/raw_50_ft_googlenet_adam/logs/train.log20170423-120901.1847 -> /storage/nets/raw_50_ft_googlenet_adam/logs/train.log


### Look at some prediction results

In [ ]:
test_net = solver.test_nets[0]
for image_index in range(10):
    gtlist = test_net.blobs['label'].data[image_index, ...].astype(np.int)
    estlist = test_net.blobs['score'].data[image_index, ...] > 0
    if np.where(estlist)[0].any():
        plt.figure()
        plt.imshow(transformer.deprocess(copy(test_net.blobs['data'].data[image_index, ...])))
        plt.title('GT: {} \n EST: {}'.format(labels[np.where(gtlist)], labels[np.where(estlist)]))
        plt.axis('off')